In [1]:
import os
import numpy as np
from tqdm import tqdm
from PIL import Image


def get_all_korean():

    def nextKorLetterFrom(letter):
        lastLetterInt = 15572643
        if not letter:
            return '가'
        a = letter
        b = a.encode('utf8')
        c = int(b.hex(), 16)

        if c == lastLetterInt:
            return False

        d = hex(c + 1)
        e = bytearray.fromhex(d[2:])

        flag = True
        while flag:
            try:
                r = e.decode('utf-8')
                flag = False
            except UnicodeDecodeError:
                c = c+1
                d = hex(c)
                e = bytearray.fromhex(d[2:])
        return e.decode()

    returns = []
    flag = True
    k = ''
    while flag:
        k = nextKorLetterFrom(k)
        if k is False:
            flag = False
        else:
            returns.append(k)
    return returns

def merge_img(img0, img1, img2):
    mmg = Image.new("L", (128*2, 128*2), 255)
    mmg.paste(img0, (0,0))
    mmg.paste(img1, (128,0))
    if img2 is not None:
        mmg.paste(img2, (64,128))
    return mmg.resize((96,96), Image.BILINEAR)

def get_imgs(prefix, font, letter):
    
    ch1_list = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    ch2_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    ch3_list = [None, 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

    hlist = [0,1,2,3,4,5,6,7,20] # ㅏㅐㅑㅒㅓㅔㅕㅖㅣ
    vlist = [8,12,13,17,18] # ㅗㅛㅜㅠㅡ
    clist = [9,10,11,14,15,16,19] # ㅘㅙㅚㅝㅞㅟㅢ
    
    
    ch1 = (ord(letter) - ord('가'))//588
    ch2 = ((ord(letter) - ord('가')) - (588*ch1)) // 28
    ch3 = (ord(letter) - ord('가')) - (588*ch1) - 28*ch2
    
    l1 = ch1_list[ch1]
    l2 = ch2_list[ch2]
    l3 = ch3_list[ch3]
    
    if ch2 in hlist:
        tag = "hparts"
    else:
        tag = "vparts"
    
    p1 = f"{prefix}_{tag}/{font}/{font}__{l1}.png"
    p1 = p1 if os.path.exists(p1) else f"{prefix}_parts/{font}/{font}__{l1}.png"
    p2 = f"{prefix}_parts/{font}/{font}__{l2}.png"
    p3 = None if l3 is None else f"{prefix}_parts/{font}/{font}__{l3}.png"

    return (
        Image.open(p1),
        Image.open(p2),
        None if p3 is None else Image.open(p3),
    )

In [2]:
fds = [
    "/home/jupyter/ai_font/data/exp0717/train0730_whole",
    "/home/jupyter/ai_font/data/exp0717/train0730_parts",
    "/home/jupyter/ai_font/data/exp0717/train0730_hparts",
    "/home/jupyter/ai_font/data/exp0717/train0730_vparts",
]

In [5]:
originalfd = "/home/jupyter/ai_font/data/exp0717/train0730_whole"
assemblefd = "/home/jupyter/ai_font/data/exp0717/train0730_assembled"
prefix = "/home/jupyter/ai_font/data/exp0717/train0730"
ak = get_all_korean()

In [6]:
for font in tqdm([f for f in os.listdir(originalfd) if ".ipy" not in f]):
    os.makedirs(f"{assemblefd}/{font}", exist_ok=True)
    for letter in ak:
        if os.path.exists(f"{originalfd}/{font}/{font}__{letter}.png"):
            img0, img1, img2 = get_imgs(prefix, font, letter)
            img = merge_img(img0,img1,img2)
            img.save(f"{assemblefd}/{font}/{font}__{letter}.png")

100%|██████████| 343/343 [51:29<00:00,  9.01s/it] 
